In [10]:
import numpy as np
import pandas as pd
from IPython.display import display
from scipy.optimize import minimize
from hmmlearn.hmm import GaussianHMM
import warnings

warnings.filterwarnings('ignore')
np.set_printoptions(precision=3)
class EMAlgorithm:
    def __init__(self, F1, F2, z, outcome, max_iterations=600, convergence_threshold=0.0001):
        self.F1 = F1
        self.F2 = F2
        self.z = z[:, 0].reshape(-1, 1)
        self.outcome = outcome
        self.max_iterations = max_iterations
        self.convergence_threshold = convergence_threshold
        self.weights = None
        self.bias = None
        self.thetas = None
        self.log_likelihoods = [100]
        self.N = len(self.z)
        self.K = 2
        self.transmat = None
        self.means = None
        self.covariance = None
        self.start_prob = None

    
    def logistic_prob_batch(self, X, weights, bias):
        return 1 / (1 + np.exp(-np.dot(X, weights) - bias))
    
    def compute_policy_batch(self, thetas, posterior_probs):
        exp_R1 = np.exp(thetas)
        pi_a1_given_s = exp_R1 / (exp_R1 + 1)
        pi_a0_given_s = 1 - pi_a1_given_s
        c1_sums = np.sum(pi_a1_given_s * posterior_probs, axis=1)
        c0_sums = np.sum(pi_a0_given_s * posterior_probs, axis=1)
        return np.column_stack((c0_sums, c1_sums))
    
    def expectation_step(self, data, K, N, weights, bias, thetas, posterior):
        mu_0 = self.logistic_prob_batch(data, weights[0], bias[0])
        mu_0 = mu_0 * self.outcome + (1 - mu_0) * (1 - self.outcome)
        mu_1 = self.logistic_prob_batch(data, weights[1], bias[1])
        mu_1 = mu_1 * self.outcome + (1 - mu_1) * (1 - self.outcome)
        policy = self.compute_policy_batch(thetas, posterior)
        denominator0, denominator1 = policy[:, 0] * mu_0, policy[:, 1] * mu_1
        sum_denominator = denominator0 + denominator1
        gamma = np.zeros((N, K))
        gamma[:, 0] = denominator0 / sum_denominator
        gamma[:, 1] = denominator1 / sum_denominator
        return gamma
    
    def maximization_step(self, data, K, N, gamma, init_b, init_c, posterior):
        def objective(params):
            new_weights, new_theta = params[:self.F1+self.F2], params[self.F1+self.F2:]
            weights = [new_weights[:self.F1], new_weights[self.F1:self.F1+self.F2]]
            thetas = new_theta
            mu_0 = self.logistic_prob_batch(data, weights[0], self.bias[0])
            mu_1 = self.logistic_prob_batch(data, weights[1], self.bias[1])
            mu_0 = mu_0 * self.outcome + (1 - mu_0) * (1 - self.outcome)
            mu_1 = mu_1 * self.outcome + (1 - mu_1) * (1 - self.outcome)
            policy = self.compute_policy_batch(thetas, posterior)
            log_likelihood = np.sum(gamma[:, 0] * np.log(policy[:, 0] * mu_0) +
                                    gamma[:, 1] * np.log(policy[:, 1] * mu_1))
            return -log_likelihood
        
        result = minimize(fun=objective, x0=np.concatenate([init_b, init_c]), method='SLSQP')
        return result.x[:self.F1+self.F2], result.x[self.F1+self.F2:]
    
    def logLikelihoodCalculation_batch(self,posteriors):
        mu_0 = self.logistic_prob_batch(self.z, self.weights[0], self.bias[0])
        mu_0 = mu_0 * self.outcome + (1 - mu_0) * (1 - self.outcome)
        mu_1 = self.logistic_prob_batch(self.z, self.weights[1], self.bias[1])
        mu_1 = mu_1 * self.outcome + (1 - mu_1) * (1 - self.outcome)
        
        policy = self.compute_policy_batch(self.thetas, posteriors)
        
        likelihood = np.zeros((self.N, self.K))
        likelihood[:, 0] = mu_0 * policy[:, 0]
        likelihood[:, 1] = mu_1 * policy[:, 1]
        
        log_likelihood = np.sum(np.log(likelihood[:, 0] + likelihood[:, 1]))
        return log_likelihood
    
    def fit(self):
        N = len(self.z)
        K = 2
        print('Starting Stage-1 Estimation..')
        best_ll, best_model = None, None
        X = self.z
        for i in range(20):
            model = GaussianHMM(K, n_iter=200, tol=1e-4, random_state=i)
            model.fit(X)
            score = model.score(X)
            if best_ll is None or best_ll < score:
                best_ll, best_model = score, model
        print('Converged Stage-1')
        self.transmat = best_model.transmat_
        self.means =best_model.means_
        self.covariance = best_model.covars_
        self.start_prob = best_model.startprob_

        posteriors = best_model.predict_proba(X)

        if np.argmax(best_model.startprob_) != 0:  
            posterior_new = best_model.predict_proba(X)
            posteriors[:, 1] = posterior_new[:, 0]
            posteriors[:, 0] = posterior_new[:, 1]

        z = self.z
        np.random.seed(0)
        init_b = np.random.random(self.F1 + self.F2)
        init_c = np.array([0, 0])
        self.weights = [init_b[:self.F1], init_b[self.F1:self.F1+self.F2]]
        self.bias = [0, 0]
        self.thetas = init_c
        best_log_likelihood = float('-inf')  # Start with negative infinity
        best_weights = None
        best_theta = None
        print('Starting Stage-2 Estimation..')
        for i in range(self.max_iterations):
            gamma = self.expectation_step(z, K, N, self.weights, self.bias, self.thetas, posteriors)
            new_weights, new_thetas = self.maximization_step(z, K, N, gamma, init_b, init_c, posteriors)
            self.weights = [new_weights[:self.F1], new_weights[self.F1:self.F1+self.F2]]
            self.thetas = new_thetas
            current_log_likelihood = self.logLikelihoodCalculation_batch(posteriors)
            self.log_likelihoods.append(current_log_likelihood)
            
            if np.isnan(current_log_likelihood):
                print("Stopped due to NaN in log-likelihood.")
                break
            
            if current_log_likelihood > best_log_likelihood:
                best_log_likelihood = current_log_likelihood
                best_weights = self.weights
                best_theta = self.thetas
                worse_count = 0
            else:
                worse_count += 1
            
            if i % 100 == 0:
                print(f"Iteration {i + 1}: Log likelihood = {current_log_likelihood}")
                print('Weights:', self.weights)
                print('Theta:', self.thetas)
            
            if abs(self.log_likelihoods[-1] - self.log_likelihoods[-2]) < self.convergence_threshold:
                print("Converged Stage-2")
                self.weights = best_weights
                self.thetas = best_theta
                break
            if worse_count >= 100:
                print("Stopped due to 10 consecutive worsening iterations.")
                break

            i+=1

        
        return self.weights, self.thetas
    
    def display_parameters(self):
        print("True Transition Probabilities:")
        true_transmat = np.array([[0.8, 0.2], 
                                [0.1, 0.9]]) 
        df_true_transmat = pd.DataFrame(true_transmat, 
                                        columns=[f"State {i}" for i in range(len(true_transmat))], 
                                        index=[f"State {i}" for i in range(len(true_transmat))])
        display(df_true_transmat.round(3))

        print("Estimated Transition Probabilities:")
        df_transmat = pd.DataFrame(self.transmat[::-1, ::-1], 
                                columns=[f"State {i}" for i in range(len(self.transmat))], 
                                index=[f"State {i}" for i in range(len(self.transmat))])
        display(df_transmat.round(3))

        print("True Means:")
        true_means = np.array([[1.0], 
                            [2.5]])  
        df_true_means = pd.DataFrame(true_means, 
                                    columns=[f"Feature {i}" for i in range(true_means.shape[1])], 
                                    index=[f"State {i}" for i in range(true_means.shape[0])])
        display(df_true_means.round(3))

        print("Estimated Means:")
        df_means = pd.DataFrame(self.means[::-1, ::-1], 
                                columns=[f"Feature {i}" for i in range(self.means.shape[1])], 
                                index=[f"State {i}" for i in range(self.means.shape[0])])
        display(df_means.round(3))


        print("True Standard deviation:")
        true_covars = np.array([[[0.03]], 
                                [[0.1]]])  
        df_true_covars = pd.DataFrame(true_covars.reshape(true_covars.shape[0], -1), 
                                    index=[f"State {i}" for i in range(true_covars.shape[0])],
                                    columns=[f"Feature {i}" for i in range(true_covars.shape[1] * true_covars.shape[2])])
        display(df_true_covars.round(3))

        print("Estimated Standard deviation:")
        df_covars = pd.DataFrame(self.covariance.reshape(self.covariance.shape[0], -1)[::-1, ::-1], 
                                index=[f"State {i}" for i in range(self.covariance.shape[0])],
                                columns=[f"Feature {i}" for i in range(self.covariance.shape[1])])
        display(df_covars.round(3))


        print("True Start Probabilities:")
        true_startprob = np.array([1.0, 0.0])  
        df_true_startprob = pd.DataFrame(true_startprob.reshape(1, -1), 
                                        columns=[f"State {i}" for i in range(len(true_startprob))])
        display(df_true_startprob.round(3))

        print("Estimated Start Probabilities:")
        df_startprob = pd.DataFrame(self.start_prob.reshape(1, -1)[::-1, ::-1], 
                                    columns=[f"State {i}" for i in range(len(self.start_prob))])
        display(df_startprob.round(3))

        # Display Estimated Weights and Theta
        print("True Weights:")
        true_weights = np.array([[0.5],[3.0]])
        df_true_weights = pd.DataFrame(true_weights.reshape(1, -1), 
                                        columns=[f"State {i}" for i in range(len(true_weights))])
        display(df_true_weights.round(3))
        print("Estimated Weights:")
        df_est_weights = pd.DataFrame(np.array(self.weights).reshape(1, -1), 
                                        columns=[f"State {i}" for i in range(len(self.weights))])
        display(df_est_weights.round(3))
        print("True Theta:")
        true_theta = np.array([-2.0,1.5])
        df_true_theta = pd.DataFrame(true_theta.reshape(1, -1), 
                                        columns=[f"State {i}" for i in range(len(true_theta))])
        display(df_true_theta.round(3))
        print("Estimated Theta:")
        df_est_theta = pd.DataFrame(np.array(self.thetas).reshape(1, -1), 
                                        columns=[f"State {i}" for i in range(len(self.thetas))])
        display(df_est_theta.round(3))

In [11]:
# Generate some sample data (Replace this with actual data)
F1, F2 = 1, 1  # Feature dimensions
z = np.load("synthetic_data_observations.npy")
outcomes = np.load("synthetic_data_outcomes.npy")

# Initialize EM-HMM model
em_hmm = EMAlgorithm(F1, F2,z, outcomes)
result = em_hmm.fit()

Starting Stage-1 Estimation..
Converged Stage-1
Starting Stage-2 Estimation..
Iteration 1: Log likelihood = -16639.227608635323
Weights: [array([1.047]), array([1.192])]
Theta: [0.001 0.054]
Iteration 101: Log likelihood = -16130.717490145236
Weights: [array([0.554]), array([2.688])]
Theta: [-2.298  1.391]
Iteration 201: Log likelihood = -16130.284216366388
Weights: [array([0.537]), array([2.954])]
Theta: [-2.267  1.423]
Iteration 301: Log likelihood = -16130.252542910373
Weights: [array([0.532]), array([3.])]
Theta: [-2.231  1.434]
Iteration 401: Log likelihood = -16130.233736399983
Weights: [array([0.528]), array([2.991])]
Theta: [-2.201  1.444]
Iteration 501: Log likelihood = -16130.21685568338
Weights: [array([0.525]), array([2.969])]
Theta: [-2.173  1.453]


In [12]:
em_hmm.display_parameters()

True Transition Probabilities:


,State 0,State 1
State 0,0.8,0.2
State 1,0.1,0.9


Estimated Transition Probabilities:


,State 0,State 1
State 0,0.793,0.207
State 1,0.099,0.901


True Means:


...

,State 0,State 1
0,0.522,2.945


True Theta:


,State 0,State 1
0,-2.0,1.5


Estimated Theta:


,State 0,State 1
0,-2.147,1.461
